importing required libraries

In [1]:
import re
import ast
import nltk
import pickle
import numpy as np
import pandas as pd
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from gensim.models import KeyedVectors
from sklearn.metrics.pairwise import cosine_similarity

pipelining the data for better efficiency of model

In [2]:
anidf = pd.read_csv(r"C:\Users\HP\Desktop\Anirec\Anirec\data\preprocessed_ani_data.csv")

In [3]:
pd.set_option('display.max_columns', None)
display.max_characters = None

In [4]:
anidf.head()

,unique_id,name_english,name,score,ranked,popularity,members,synopsis,synonyms,type_of,total_episodes,premiered,studios,genres,demographic,duration_per_ep,rating,scored_by,favorites,aired,source,watching,completed,on_hold,dropped,plan_to_watch,total,scored_10_by,scored_9_by,scored_8_by,scored_7_by,scored_6_by,scored_5_by,scored_4_by,scored_3_by,scored_2_by,scored_1_by
0,0,Tomorrow's Joe 2,Ashita no Joe 2,8.73,50,3041,51773,Yabuki Joe is left downhearted and hopeless af...,Rocky Joe 2,TV,47.0,Fall 1980,Tokyo Movie Shinsha,"['Historical', 'Sports', 'Josei', 'Drama', 'Ad...",Shounen,24 min. per ep.,PG-13 - Teens 13 or older,20201.0,2653,"Oct 13, 1980 to Aug 31, 1981",Manga,2723,23933,1105,"1,205",22807,51773,7721.0,4711.0,3690.0,1797.0,653.0,297.0,120.0,137.0,182.0,893.0
1,1,Mushi-shi: Next Passage Part 2,Mushishi Zoku Shou 2nd Season,8.73,49,907,265728,"Ghostly, primordial beings known as Mushi cont...",Mushishi Zoku Shou 2nd Season,TV,10.0,Fall 2014,Artland,"['Supernatural', 'Slice of Life', 'Action', 'A...",Seinen,23 min. per ep.,PG-13 - Teens 13 or older,106197.0,1475,"Oct 19, 2014 to Dec 21, 2014",Manga,14438,133486,6536,"3,332",107936,265728,30834.0,34334.0,26218.0,9892.0,2672.0,974.0,283.0,134.0,183.0,673.0
2,2,Kingdom: Season 4,Kingdom 4th Season,8.74,48,2731,63418,Following the conclusion of the large-scale co...,キングダム 第4シリーズ,TV,26.0,Spring 2022,Pierrot,"['Historical', 'Sports', 'Action', 'Drama', 'S...",Seinen,24 min. per ep.,R - 17+ (violence & profanity),26633.0,1043,"Apr 10, 2022 to Oct 2, 2022",Manga,8497,29630,1084,688,23519,63418,7524.0,9439.0,6590.0,1916.0,461.0,163.0,68.0,52.0,139.0,281.0
3,3,Vinland Saga,Vinland Saga,8.75,47,77,1502303,Young Thorfinn grew up listening to the storie...,ヴィンランド・サガ,TV,24.0,Summer 2019,Wit Studio,"['Fantasy', 'Historical', 'Adventure', 'Action...",Seinen,24 min. per ep.,R - 17+ (violence & profanity),863875.0,47017,"Jul 8, 2019 to Dec 30, 2019",Manga,137792,1002257,39747,"22,421",300086,1502303,228893.0,326376.0,209929.0,67519.0,17475.0,6602.0,2908.0,1321.0,1030.0,1822.0
4,4,Cowboy Bebop,Cowboy Bebop,8.75,46,43,1857971,"Crime is timeless. By the year 2071, humanity ...",カウボーイビバップ,TV,26.0,Spring 1998,Sunrise,"['Seinen', 'Action', 'Award Winning', 'Science...",NaN,24 min. per ep.,R - 17+ (violence & profanity),960699.0,82113,"Apr 3, 1998 to Apr 24, 1999",Original,172491,1073071,106423,"42,996",462990,1857971,332862.0,271436.0,203324.0,96823.0,32270.0,13593.0,4994.0,1997.0,1120.0,2280.0


In [5]:
anidf['demographic'] = anidf['demographic'].apply(lambda x: [x])

In [6]:
anidf['demographic'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 14740 entries, 0 to 14739
Series name: demographic
Non-Null Count  Dtype 
--------------  ----- 
14740 non-null  object
dtypes: object(1)
memory usage: 115.3+ KB


In [7]:
type(anidf['genres'][1])

str

In [8]:
anidf['genres'][1]

"['Supernatural', 'Slice of Life', 'Action', 'Adventure', 'Mystery']"

In [9]:
def convert_to_listg(x):
    try:
        return ', '.join(ast.literal_eval(x))
    except (ValueError, SyntaxError, TypeError):
        return np.nan

anidf['genres'] = anidf['genres'].apply(convert_to_listg)

In [10]:
anidf['genres'][1]

'Supernatural, Slice of Life, Action, Adventure, Mystery'

In [11]:
def convert_to_list_d(x):
    if isinstance(x, list):
        return ', '.join(map(str, x))
    else:
        return ' '
    
anidf['demographic'] = anidf['demographic'].apply(convert_to_list_d)


In [12]:
anidf['demographic'].head()

0    Shounen
1     Seinen
2     Seinen
3     Seinen
4        nan
Name: demographic, dtype: object

In [13]:
anidf['genres'] = anidf['genres'].fillna('')
anidf['demographic'] = anidf['demographic'].fillna('')

In [14]:
anidf['combined_text'] = anidf['synopsis'] + anidf['genres'] + anidf['demographic'] + '.'

In [15]:
type(anidf['combined_text'][1])

str

In [16]:
model = KeyedVectors.load_word2vec_format(r"C:\Users\HP\Desktop\Anirec\Anirec\data\word2vec_model.bin", binary=True)

In [17]:
def preprocess_text(text):
    # Convert text to lowercase
    text = text.lower()
    
    # Remove numbers
    text = re.sub('\d+', '', text)
    
    # Remove punctuation
    text = re.sub('[^\w\s]', '', text)
    
    # Tokenize the text into words
    words = word_tokenize(text)
    
    # Remove stopwords
    words = [word for word in words if word not in stopwords.words('english')]
    
    return words

def text_to_vector(input_text, model):

    words_list = preprocess_text(input_text)
    vectors = [model[word] for word in words_list if word in model.key_to_index]
    
    if vectors:
        avg_vector = np.mean(vectors, axis=0)
    else:
        avg_vector = np.zeros(model.vector_size)
    
    return avg_vector

In [18]:
anidf['aniVector'] = anidf['combined_text'].apply(lambda x: text_to_vector(x, model))

In [19]:
vectors = np.array(anidf['aniVector'].tolist())

In [20]:
similarity_matrix = cosine_similarity(vectors, vectors)

In [21]:
similarity_matrix

array([[0.99999994, 0.8652617 , 0.8453286 , ..., 0.5755793 , 0.8746348 ,
        0.8827986 ],
       [0.8652617 , 0.9999999 , 0.85003555, ..., 0.6029074 , 0.87277126,
        0.9086808 ],
       [0.8453286 , 0.85003555, 0.9999998 , ..., 0.49591064, 0.73872405,
        0.8420339 ],
       ...,
       [0.5755793 , 0.6029074 , 0.49591064, ..., 1.        , 0.63156366,
        0.6284406 ],
       [0.8746348 , 0.87277126, 0.73872405, ..., 0.63156366, 0.99999994,
        0.8783954 ],
       [0.8827986 , 0.9086808 , 0.8420339 , ..., 0.6284406 , 0.8783954 ,
        1.0000001 ]], dtype=float32)

**The main recommendation function**

In [22]:
def recommend(anime, top=3):
    if anime not in anidf['name_english'].values:
        return f"Anime '{anime}' not found in the dataset."
    
    anime_index = anidf[anidf['name_english'] == anime].index[0]
    distances = similarity_matrix[anime_index]
    
    anime_list = [(i, dist) for i, dist in enumerate(distances) if anidf.iloc[i].name_english != anime]
    anime_list = sorted(anime_list, key=lambda x: x[1], reverse=True)[:top+10]
    
    recommended_animes = [anidf.iloc[i[0]].name_english for i in anime_list]
    recommended_animes = list(set(recommended_animes))
    
    pattern = r'\b' + re.escape(anime) + r'\b'
    recommended_animes = [title for title in recommended_animes if not re.search(pattern, title, re.IGNORECASE)]
    
    return recommended_animes[:top] if len(recommended_animes) >= top else recommended_anime

In [23]:
recommend('Fairy Tail', top = 10)

['Goblin Slayer',
 'Arad Senki: Slap Up Party',
 'Magi: The Labyrinth of Magic',
 'Huan Bian Jingling: Dangao Tianxin',
 'Magi: Sinbad no Bouken',
 "Chillin' in Another World with Level 2 Super Cheat Powers",
 'The Great Adventures of Robin Hood',
 'Slayers',
 'Maho Girls Precure!',
 'Dorohedoro']

In [24]:
with open(r'C:\Users\HP\Desktop\Anirec\Anirec\data\similarity_matrix.pkl', 'wb') as f:
    pickle.dump(similarity_matrix, f)